In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from funcs import read_config, get_groups, pull_clean, get_kpi2_respuestas, get_kpi3_retencion, get_kpi4_influencia

In [2]:
business_yelpp = pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz')
business_yelpp

,business_id,name,latitude,longitude,avg_stars,state_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.09,1
1,0bPLkL0QhhPO5kt1_EXmNQ,Zios Italian Market,27.916116,-82.760461,4.45,1
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.00,1
3,JgpnXv_0XhV3SfbfB50nxw,Joes Pizza,27.960514,-82.506127,4.17,1
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.51,1
...,...,...,...,...,...,...
12912,Df-DdXqssj7ZeD8ypAF7Og,Crux Cafe,39.894469,-75.071001,4.00,4
12913,qjtELTt9fdIwoi_xGNN21g,Chipotle Mexican Grill,39.943481,-74.964743,2.43,4
12914,BzPTw9daJW8ToqTc7QiHFw,Taco Bell,40.210771,-74.756588,2.67,4
12915,UBQAksw81m0sMrAd8g-ECg,El Zarape Restaurant,39.718717,-74.971143,4.00,4


In [5]:
#Data Pull and Functions
data_frames = pull_clean() 

state = data_frames.get('1_states.parquet')
business_google = data_frames.get('5_business_google.parquet')
business_yelp = data_frames.get('6_business_yelp.parquet')


db_route
1_states.parquet: (4, 2)
5_business_google.parquet: (24801, 6)
6_business_yelp.parquet: (12917, 6)


In [7]:
business_yelp

,business_id,name,latitude,longitude,avg_stars,state_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.09,1
1,0bPLkL0QhhPO5kt1_EXmNQ,Zios Italian Market,27.916116,-82.760461,4.45,1
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.00,1
3,JgpnXv_0XhV3SfbfB50nxw,Joes Pizza,27.960514,-82.506127,4.17,1
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.51,1
...,...,...,...,...,...,...
12912,Df-DdXqssj7ZeD8ypAF7Og,Crux Cafe,39.894469,-75.071001,4.00,4
12913,qjtELTt9fdIwoi_xGNN21g,Chipotle Mexican Grill,39.943481,-74.964743,2.43,4
12914,BzPTw9daJW8ToqTc7QiHFw,Taco Bell,40.210771,-74.756588,2.67,4
12915,UBQAksw81m0sMrAd8g-ECg,El Zarape Restaurant,39.718717,-74.971143,4.00,4


In [ ]:
business_yelp[business_yelp['name'].isnull()]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
groups = get_groups(business_yelp)

groups